<br>

### `SelfQueryRetriever`
- **메타데이터 필드 와 문서 내용에 대한 간단한 설명을 미리 제공**
- `AttributeInfo` : **메타데이터 필드에 대한 정보를 정의**

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chains.query_constructor.base import AttributeInfo

In [3]:
metadata_field_info = [
    AttributeInfo(
        name="category",
        description="The category of the cosmetic product. One of ['스킨케어', '메이크업', '클렌징', '선케어']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the cosmetic product was released",
        type="integer",
    ),
    AttributeInfo(
        name="user_rating",
        description="A user rating for the cosmetic product, ranging from 1 to 5",
        type="float",
    ),
]

<br>

- **리트리버 생성**

In [4]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
)

<br>

#### Query 테스트

In [ ]:
retriever.invoke("평점이 4.8 이상인 제품을 추천해주세요")

[Document(id='942f859b-2236-4d3b-a1f8-36418609605e', metadata={'year': 2024, 'user_rating': 4.9, 'category': '선케어'}, page_content='자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.'),
 Document(id='8fed47fd-b2ec-418e-a3dd-5030b1af7636', metadata={'category': '클렌징', 'year': 2023, 'user_rating': 4.8}, page_content='식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.')]

In [ ]:
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

[Document(id='06a60b0c-1ac9-49c7-a4ee-885fc9bc1632', metadata={'category': '메이크업', 'year': 2023, 'user_rating': 4.5}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.'),
 Document(id='72b5848e-89d4-4a7c-8d42-874ebfde399e', metadata={'category': '스킨케어', 'user_rating': 4.6, 'year': 2023}, page_content='비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.'),
 Document(id='8fed47fd-b2ec-418e-a3dd-5030b1af7636', metadata={'year': 2023, 'user_rating': 4.8, 'category': '클렌징'}, page_content='식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.')]

In [ ]:
retriever.invoke("카테고리가 선케어인 상품을 추천해주세요")

[Document(id='942f859b-2236-4d3b-a1f8-36418609605e', metadata={'year': 2024, 'user_rating': 4.9, 'category': '선케어'}, page_content='자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.')]

In [ ]:
retriever.invoke(
    "카테고리가 메이크업인 상품 중에서 평점이 4.5 이상인 상품을 추천해주세요"
)

[Document(id='06a60b0c-1ac9-49c7-a4ee-885fc9bc1632', metadata={'category': '메이크업', 'year': 2023, 'user_rating': 4.5}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.')]

<br>

- **가져올 문서의 수 지정 :`k`**

In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 2}, 
)

In [ ]:
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

[Document(id='06a60b0c-1ac9-49c7-a4ee-885fc9bc1632', metadata={'user_rating': 4.5, 'year': 2023, 'category': '메이크업'}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.'),
 Document(id='72b5848e-89d4-4a7c-8d42-874ebfde399e', metadata={'year': 2023, 'category': '스킨케어', 'user_rating': 4.6}, page_content='비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.')]

<br>

### 구조화된 쿼리 체인 (`query_constructor`)
- **Self-query retriever의 핵심 요소는 `query constructor`**

In [ ]:
from langchain.chains.query_constructor.base import StructuredQueryOutputParser, get_query_constructor_prompt

- **쿼리 생성기 프롬프트**

In [ ]:
prompt = get_query_constructor_prompt(
    "Brief summary of a cosmetic product",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보
)

- 출력 파서

In [ ]:
output_parser = StructuredQueryOutputParser.from_components()

- 체인 생성

In [ ]:
query_constructor = prompt | llm | output_parser

<br>

- 생성된 쿼리 확인

In [ ]:
query_output = query_constructor.invoke(
    {
        "query": "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
    }
)

In [ ]:
query_output.filter.arguments

[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='year', value=2023),
 Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='user_rating', value=4.5),
 Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='category', value='스킨케어')]

<br>

### 구조화된 쿼리 변환기(Structured Query Translator)
* **이는 일반적인 `StructuredQuery` 객체를 사용 중인 vector store의 구문에 맞는 메타데이터 필터로 변환**

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

In [ ]:
retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # 이전에 생성한 query_constructor chain 을 지정
    vectorstore=vectorstore,  # 벡터 저장소를 지정
    structured_query_translator=ChromaTranslator(),  # 쿼리 변환기
)

In [ ]:
retriever.invoke(
    "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
)

[Document(id='72b5848e-89d4-4a7c-8d42-874ebfde399e', metadata={'category': '스킨케어', 'year': 2023, 'user_rating': 4.6}, page_content='비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.')]

<br>

<hr>

<br>

## 시간 가중 벡터저장소 리트리버(`TimeWeightedVectorStoreRetriever`)
- **의미론적 유사성과 시간에 따른 감쇠를 결합해 사용하는 검색 도구**
  
  $\rightarrow$ **문서 또는 데이터의 "신선함" 과 "관련성" 을 모두 고려하여 결과를 제공**

<br>


- **스코어링 알고리즘**
  - `semantic_similarity` : 문서 또는 데이터 간의 의미적 유사도
  - `decay_rate` : 시간이 지남에 따라 점수가 얼마나 감소하는지를 나타내는 비율
  - `hours_passed`: 객체가 마지막으로 접근된 후부터 현재까지 경과한 시간(시간 단위)

<math xmlns="http://www.w3.org/1998/Math/MathML">
  <mtext>semantic_similarity</mtext>
  <mo>+</mo>
  <mo stretchy="false">(</mo>
  <mn>1.0</mn>
  <mo>&#x2212;</mo>
  <mtext>decay_rate</mtext>
  <msup>
    <mo stretchy="false">)</mo>
    <mrow data-mjx-texclass="ORD">
      <mi>h</mi>
      <mi>o</mi>
      <mi>u</mi>
      <mi>r</mi>
      <msub>
        <mi>s</mi>
        <mi>p</mi>
      </msub>
      <mi>a</mi>
      <mi>s</mi>
      <mi>s</mi>
      <mi>e</mi>
      <mi>d</mi>
    </mrow>
  </msup>
</math>

<br>

- 객체가 마지막으로 접근된 시간을 기준으로 하여 "정보의 신선함" 을 평가
  
  $\rightarrow$ **자주 접근되는 객체는 시간이 지나도 높은 점수를 유지하며, 이를 통해 자주 사용되거나 중요하게 여겨지는 정보가 검색 결과 상위에 위치할 가능성이 높아짐**
  
- 이런 방식은 최신성과 관련성을 모두 고려하는 동적인 검색 결과를 제공하며, `decay_rate` 는 **리트리버의 객체가 생성된 이후가 아니라 마지막으로 액세스된 이후 경과된 시간을 의미**

  $\rightarrow$ **자주 액세스하는 객체는 '최신'으로 유지**

<br>

### 낮은 감쇠율(low decay_rate)

In [7]:
from datetime import datetime, timedelta

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [8]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 초기화합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

- 시간 가중치가 적용된 벡터 저장소 검색기를 초기화

In [14]:
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=1e-10, k=1
)

- 예제 데이터

In [15]:
yesterday = datetime.now() - timedelta(days=1)

retriever.add_documents(
    [
        Document(
            page_content="테디노트 구독해 주세요.",
            metadata={"last_accessed_at": yesterday},
        ),
        Document(
            page_content="테디노트 구독 해주실꺼죠? Please!"
        )
    ]
)

['9cde131a-80d2-41cf-a769-c3692a046d93',
 '190a65f3-42dc-454b-a3d4-fbe247d9e48f']

- **낮은 감쇠율로 인하여, 벡터 스토어에 먼저 들어간 입력데이터의 최신성이 절하되지 않음**
  
  $\rightarrow$ 처음 들어간 문서가 출력

In [16]:
retriever.invoke("테디노트")

[Document(metadata={'last_accessed_at': datetime.datetime(2025, 11, 24, 13, 27, 57, 306416), 'created_at': datetime.datetime(2025, 11, 24, 13, 27, 55, 262055), 'buffer_idx': 0}, page_content='테디노트 구독해 주세요.')]

<br>

### 높음 감쇠율(high decay_rate)

- **높은 감쇠율은 최신성 점수가 빠르게 0으로 수렴**

In [17]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.999, k=1
)

In [19]:
yesterday = datetime.now() - timedelta(days=1)

retriever.add_documents(
    [
        Document(
            page_content="테디노트 구독해 주세요.",
            metadata={"last_accessed_at": yesterday},
        ),
        Document(
            page_content="테디노트 구독 해주실꺼죠? Please!"
        )
    ]
)

['973b6e4e-8a67-4728-bb9c-6a90a9c48399',
 '99b5d94b-ed59-419a-8e6c-aaab7a299c88']

- **낮은 감쇠율로 인하여, 벡터 스토어에 먼저 들어간 입력데이터의 최신성이 절하**
  
  $\rightarrow$ 마지막에 들어간 문서가 출력

In [20]:
retriever.invoke("테디노트")

[Document(metadata={'last_accessed_at': datetime.datetime(2025, 11, 24, 13, 31, 14, 156252), 'created_at': datetime.datetime(2025, 11, 24, 13, 31, 8, 257829), 'buffer_idx': 3}, page_content='테디노트 구독 해주실꺼죠? Please!')]

<br>

### 감쇠율(decay_rate)
- **`decay_rate` 를 매우 작게 설정한 경우, 감쇠율(즉, 정보를 망각하는 비율)이 매우 낮기 때문에 정보를 거의 잊지 않음**
  
  $\rightarrow$ **최신 정보이든 오래된 정보든 시간 가중치 차이가 거의 없습니다. 이럴때는 유사도에 더 높은 점수**

- **`decay_rate` 를 매우 높게 설정한 경우, 감쇠율(즉, 정보를 망각하는 비율)이 매우 높기 때문에 정보를 거의 다 잊어버림**
  
  $\rightarrow$ **최신 정보에 더 높은 점수**

<br>

#### 감쇠율 조정
- 시간 구성 요소를 모의(mock) 테스트

In [21]:
import datetime
from langchain.utils import mock_now

- `mock_now` 함수를 사용하여 현재 시간을 변경하면서 검색 결과를 테스트
  - 만약 너무 오래전의 시간으로 설정하면, `decay_rate` 계산시 오류가 발생

In [30]:
with mock_now(datetime.datetime(2025, 11, 23, 00, 00)):
    print(retriever.invoke("테디노트"))

[Document(metadata={'last_accessed_at': MockDateTime(2025, 11, 23, 0, 0), 'created_at': datetime.datetime(2025, 11, 24, 13, 31, 8, 257829), 'buffer_idx': 3}, page_content='테디노트 구독 해주실꺼죠? Please!')]


<br>

<hr>